# TP Reduction des dimensions : PCA et Clustering
Récupérer les données sur https://www.kaggle.com/karangadiya/fifa19

In [ ]:
import numpy as np
import pandas as pd
# Affichage complet des dataframes
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)

import itertools
import io

# Librairie graphique plotly
import plotly.graph_objs as go


## Data Preparation

In [ ]:
df=pd.read_csv("data/data.csv")
df.head(10)

In [ ]:
attaquants=['RF', 'ST', 'LW', 'LF', 'RS', 'LS', 'RM', 'LM','RW']
milieux=['RCM','LCM','LDM','CAM','CDM','LAM','RDM','CM','RAM','CF']
defenseurs=['RCB','CB','LCB','LB','RB','RWB','LWB']
gardiens=['GK']

def pos2(position):
    if position in attaquants:
        return 'Attaquant'    
    elif position in milieux:
        return 'Milieu'    
    elif position in defenseurs:
        return 'Defenseur'    
    elif position in gardiens:
        return 'Gardien'    
    else:
        return 'nan'
df["Position2"]=df["Position"].apply(lambda x: pos2(x))
df["Position2"].value_counts()

In [ ]:
best_players_per_position=df.iloc[df.groupby(df['Position2'])['Overall'].idxmax()][['Position2','Name','Overall']]
best_players_per_position

## T-SNE

In [ ]:
n_sne=2000 # Limitation au 2000 premiers joueurs
df_sne = df.loc[:n_sne]
competences_ratings = ['Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
                  'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
                  'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
                  'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
                  'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 
                  'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
                  'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

# 1.1 Construire Un dataframe des compétences 
X=df_sne[competences_ratings]
X.sample(10)

In [ ]:
#1.2 Exécuter une transformation T_SNE sur les données 


In [ ]:
#1.3 Créer des filtres booléens par type de poste, puis sur les meilleurs joueurs (Overall au dessus de 90)


In [ ]:
palette = ['navy','red','maroon','orange','green','pink']

#1.4 Creer 4 nuages de points correspondant aux attaquants, milieux, défenseurs et gradient.

#1.5 Ajouter les noms des meilleurs joueurs


## ACP + Clustering

In [ ]:
n_kmean=18000

# Suppression des gardiens de but

df_kmean=df.loc[:n_kmean]
df_kmean=df_kmean[(df_kmean["Position"]!='GK')&(df_kmean['Overall']>69)]

competences_ratings = ['Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                  'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
                  'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions',
                  'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']

# Création d'un joueur parfait pour savoir où il se situe

JoueurParfaitDict={'Name':'MrParfait','Overall':99}
for competence in competences_ratings:
    JoueurParfaitDict[competence]=99
    
df_kmean=df_kmean.append(JoueurParfaitDict,ignore_index=True)

df_competences=df_kmean[competences_ratings]
X=df_competences
X

In [ ]:
#2.1 Créer une division en 5 clusters des joueurs


In [ ]:
#2.2 Appliquer une PCA à 2 dimensions sur les joueurs

In [ ]:
#2.3 Afficher les clusters en couleurs sur la représentation PCA en 2D des joueurs. 
# Ajouter un label pour les meilleurs joueurs (Overall au dessus de 90)

filtre_meilleurs = df_kmean["Overall"]>90
kmeans_clusters=go.Scatter(x=X_projected[:,0],y=X_projected[:,1],mode='markers',
                           marker=dict(
                                size=5,
                                color=clusters.astype(np.float),
                                colorscale='Portland',
                                showscale=False
                           )
                        )
acp_meilleurs=go.Scatter(x=X_projected[filtre_meilleurs,0], y=X_projected[filtre_meilleurs,1],name='Meilleurs joueurs',
                      text=df_kmean.loc[filtre_meilleurs,'Name'],textfont=dict(family='sans serif',color='black',size=16),
                      opacity=0.9,mode='text'
                            )
data=[kmeans_clusters,acp_meilleurs]

layout = go.Layout(title="ACP + Clustering ",titlefont=dict(size=40),
                   xaxis=dict(title='F1'),
                   yaxis=dict(title='F2'),
                   autosize=False, width=1000,height=1000)

fig = go.Figure(data=data, layout=layout)

fig.show()

## 3. Qui aurait gagné la Ligue des Champions 2020 ?
Rappel des équipes encore en compétition : Paris Saint-Germain, Real Madrid, Bayern Munich, Manchester City, Atalanta Bergame, Juventus FC, Atlético de Madrid,SSC Naples,FC Barcelone,RB Leipzig, Olympique lyonnais, Chelsea FC

